In [4]:
import pickle
import gradio as gr
# load model to use
model_pkl = './models/first_binary_model.pkl'
  
# load model from pickle file
with open(model_pkl, 'rb') as file:  
    model = pickle.load(file)  
from combined_gui import predict
weapon_list = [1,0]
print(weapon_list)
predict(model, (64, 60))

[1, 0]
Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


1/1 [==============================] - 0s 95ms/step
1
1/1 [==============================] - 0s 23ms/step
